## Read Labelled .csv from Phase 2

In [9]:
import pandas as pd
job_data = pd.read_csv('linkedin_job_postings_label.csv')

In [10]:
from sklearn.model_selection import train_test_split

jobSkills = []
for i in job_data['job_skills']:
    jobSkills.append(i.lower())

Xclass = vectorizer.fit_transform(jobSkills)

X_train, X_test, y_train, y_test = train_test_split(Xclass,job_data['Label'],test_size=0.2,random_state=42)

## Best model implementation - SVM

In [11]:
from sklearn import svm
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
import matplotlib.pyplot as plt

Csvm = [0.1,0.5,0.8,1,1.5,2,2.5,3,3.5] 
trainAcc = []
testAcc = []

for i in Csvm:
    modelsvm = svm.SVC(C=i, gamma=1, kernel= 'rbf')
    svmfit = modelsvm.fit(X_train, y_train)
    svm_predtrain = modelsvm.predict(X_train)
    svm_predtest = modelsvm.predict(X_test)
    trainacc = accuracy_score(y_train, svm_predtrain)
    testacc = accuracy_score(y_test, svm_predtest)
    trainAcc.append(trainacc)
    testAcc.append(testacc)

In [12]:
svm = svm.SVC(C=5, gamma=1, kernel= 'rbf', probability=True)
svmfit = svm.fit(X_train,y_train)
svm_pred = svmfit.predict(X_test)
svm_acc = accuracy_score(y_test,svm_pred)

## Job Recommendation based on user skills

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

userInput = input("Enter your skills: ")

pred = vectorizer.transform([userInput.lower()])

output = svm.predict(pred)

print("You may look into " + output[0] + " jobs")

print("Here is a list of jobs that is under "+ output[0])
cos=[]
labelData = job_data[job_data['Label'] == output[0]].copy()

for index, row in labelData.iterrows(): 
    skills= [row['job_skills']]

    skillVec = vectorizer.transform(skills)

    cos_lib=cosine_similarity (skillVec, pred)

    cos.append( cos_lib[0][0])

labelData.loc[:, 'cosine_similarity'] = cos

#Display top 5 recommendation from cosine similarity

top_5 =labelData.sort_values('cosine_similarity', ascending=False) [['Label', 'job_title', 'company_name', 'job_city', 'job_state', 'job_skills',
       'job_level', 'job_post_link', 'last_modified_date',
       'first_seen', 'job_country']]

top_5.head(5)

Enter your skills: nurse
You may look into Healthcare Worker jobs
Here is a list of jobs that is under Healthcare Worker


,Label,job_title,company_name,job_city,job_state,job_skills,job_level,job_post_link,last_modified_date,first_seen,job_country
335,Healthcare Worker,Registered Nurse – Emergency Room - Travel - (...,TravelNurseSource,Southaven,MS,"Emergency Room Nurse, Emergency Nurse, Trauma ...",Mid senior,https://www.linkedin.com/jobs/view/registered-...,24/01/19,13-01-2024,United States
49,Healthcare Worker,RN House Supervisor,Providence Health & Services,Mission Viejo,CA,"Nursing, Patient Flow Management, Patient Plac...",Mid senior,https://www.linkedin.com/jobs/view/rn-house-su...,24/01/19,13-01-2024,United States
305,Healthcare Worker,Registered Nurse - Critical Care Neurosciences,Health eCareers,Arlington Heights,IL,"Registered Nurse, Critical Care Unit, Neurosci...",Mid senior,https://www.linkedin.com/jobs/view/registered-...,24/01/19,13-01-2024,United States
826,Healthcare Worker,RN/LPN | Nights,Redstone,Murrysville,PA,"Registered Nurse, Practical Nurse, Nursing, Me...",Mid senior,https://www.linkedin.com/jobs/view/rn-lpn-nigh...,24/01/19,13-01-2024,United States
576,Healthcare Worker,Hospice Inpatient Registered Nurse (RN) Nights...,Resource 1 Homecare Staffing,Wilmington,DE,"Registered Nurse, Hospice Nurse, Critical Care...",Associate,https://www.linkedin.com/jobs/view/hospice-inp...,24/01/19,14-01-2024,United States


## Creating .pkl files for Flask web application

In [14]:
import pickle

# Serialize (pickle) the vectorizer and svm model
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)
    
with open('svm_model.pkl', 'wb') as f:
    pickle.dump(svm, f)

with open('job_data.pkl', 'wb') as f:
    pickle.dump(job_data, f)